In [1]:
from typing import List
from random import random
import copy
import math

In [2]:
class Product:
    def __init__(self, name:str, space:float, price:float) -> None:
        '''
        Product class is analogous to a gene in Genetic Algorithm.
        '''
        self.name = name
        self.space = space
        self.price = price
        
    def __repr__(self) -> str:
        return f"{self.name} - {self.price} - {self.price}"

In [3]:
products_list = [
    Product('Refrigerator A', 0.751, 999.90),
    Product('Cell phone', 0.00000899, 2199.12),
    Product('TV 55', 0.400, 4346.99),
    Product("TV 50' ", 0.290, 3999.90),
    Product("TV 42' ", 0.200, 2999.00),
    Product("Notebook A", 0.00350, 2499.90),
    Product("Ventilator", 0.496, 199.90),
    Product("Microwave A", 0.0424, 308.66),
    Product("Microwave B", 0.0544, 429.90),
    Product("Microwave C", 0.0319, 299.29),
    Product("Refrigerator B", 0.635, 849.00),
    Product("Refrigerator C", 0.870, 1199.89),
    Product("Notebook B", 0.498, 1999.90),
    Product("Notebook C", 0.527, 3999.00),
]
products_list

[Refrigerator A - 999.9 - 999.9,
 Cell phone - 2199.12 - 2199.12,
 TV 55 - 4346.99 - 4346.99,
 TV 50'  - 3999.9 - 3999.9,
 TV 42'  - 2999.0 - 2999.0,
 Notebook A - 2499.9 - 2499.9,
 Ventilator - 199.9 - 199.9,
 Microwave A - 308.66 - 308.66,
 Microwave B - 429.9 - 429.9,
 Microwave C - 299.29 - 299.29,
 Refrigerator B - 849.0 - 849.0,
 Refrigerator C - 1199.89 - 1199.89,
 Notebook B - 1999.9 - 1999.9,
 Notebook C - 3999.0 - 3999.0]

In [4]:
class Cart:
    pass

class Cart:
    def __init__(self, products:List[Product], space_limit:float, generation:int=0) -> None:
        '''
        Cart class is analogous to a chromosome (i.e., solution) in Genetic Algorithm.
        '''
        self.products = products
        self.space_limit = space_limit
        self.generation = generation
        self.chromosome = self._init_chromosome()
        
        self._selected_products = None
        
        self._repr = None
        self.evaluated_score = 0
        self.used_space = 0
        self._update_cart_info()
        
    def _init_chromosome(self) -> List[int]:
        chromosome = []
        for _ in range(len(self.products)):
            if random() > 0.5:
                chromosome.append(0)
            else:
                chromosome.append(1)
        
        return chromosome
    
    def _create_next_generation_child_with_new_chromose(self, new_chromose:List[int]) -> Cart:
        child = Cart(self.products, self.space_limit, self.generation + 1)
        child.chromosome = new_chromose
        child._update_cart_info()
        
        return child
    
    def __repr__(self) -> str:
        return self._repr
        
    
    def __gt__(self, _other_cart:Cart) -> bool:
        return self.evaluated_score > _other_cart.evaluated_score
    
    def __eq__(self, _other_cart:Cart) -> bool:
        return self.selected_products == _other_cart.selected_products


    @property
    def selected_products(self) -> List[Product]:
        return self._selected_products

    
    def _update_selected_products(self) -> None:
        selected_products_list = []
        for gene, product  in zip(self.chromosome, self.products):
            if gene == 1:
                selected_products_list.append(product)
                
        self._selected_products = selected_products_list
        
    def _update_repr(self) -> None:
        str_repr = ""
        line_seperator = "---------------------------------------------\n"
        str_repr += line_seperator 
        for idx, product in enumerate(self.selected_products):
            product_desc = f"Product {idx+1}: {repr(product)} \n"
            str_repr += product_desc
        
        str_repr += f"Generation: {self.generation} \n"
        str_repr += f"Chromosome: {self.chromosome} \n"
        str_repr += f"Score: {self.evaluated_score} \n"
        str_repr += line_seperator
    
        self._repr = str_repr
    
    def _update_fitness(self) -> None:
        total_score = 0
        total_space = 0
        for product in self.selected_products:
            total_score += product.price
            total_space += product.space
            
        if total_space > self.space_limit:
            total_score = 1
        
        self.evaluated_score = total_score
        self.used_space = total_space
        
    def _update_cart_info(self) -> None:
        self._update_selected_products()
        self._update_fitness()
        self._update_repr()
    
        
    
    def crossover(self, _other_cart:Cart) -> List[Cart]:
        cut_off_point = math.floor(random() * len(self.chromosome))
#         print(cut_off_point)

        self_genes_x, self_genes_y = self.chromosome[:cut_off_point], self.chromosome[cut_off_point:]
        other_genes_x, other_genes_y = _other_cart.chromosome[:cut_off_point],  _other_cart.chromosome[cut_off_point:]  
        
        child_x_chromosome = [*self_genes_x, *other_genes_y]
        child_y_chromosome = [*other_genes_x, *self_genes_y]
        
#         print(child_x_chromosome, "\n", child_y_chromosome)
        
        
        next_generation_child_x = self._create_next_generation_child_with_new_chromose(child_x_chromosome)
        next_generation_child_y = self._create_next_generation_child_with_new_chromose(child_y_chromosome)
        
        next_generation_children = [
            next_generation_child_x,
            next_generation_child_y
        ]
        
        return next_generation_children
    
    def _mutate_a_gene_in_chromosome(self, idx:int) -> None:
        current_bit = self.chromosome[idx]
        new_bit = 1 if current_bit == 0 else 0
        self.chromosome[idx] = new_bit
    
    def mutation(self, mutation_rate:float) -> Cart:
#         print(f"Before: {self.chromosome}")
        
        is_mutated = False
        for idx in range(len(self.chromosome)):
            mutation_prob = random()
            if mutation_prob > mutation_rate:
                continue
                
            self._mutate_a_gene_in_chromosome(idx)
            is_mutated = True
            
        if is_mutated:
            self._update_cart_info()
            
#         print(f"After: {self.chromosome}")
        return self
            
            
                
    
    
        
            
            
            
        

In [5]:
class GeneticAlgorithm():
    def __init__(self, population_size:int, mutation_prob, num_of_generation,products:List[Product], space_limit:int) -> None:
        self.population_size = population_size
        self.population = [].copy()
        self.generation = 0
        self.num_of_generation = num_of_generation
        self.best_solution = None
        self.global_best = None
        self.mutation_prob = mutation_prob
        
        self.products = products
        self.space_limit = space_limit
        
        
    def _initialize_population(self) -> None:
        for _ in range(self.population_size):
            one_solution = Cart(self.products, self.space_limit)
            self.population.append(one_solution)
        self.population.sort(reverse = True)
        self.best_solution = self.population[0]
        self.global_best = self.best_solution
        
        
    def _total_fitness_of_population(self) -> int:
        '''
        helper function for selection (_select_parent), simulating roulette selection
        '''
        total_fitness = 0
        for cart in self.population:
            total_fitness += cart.evaluated_score
            
        return total_fitness
    
    
    def _select_partent(self) -> Cart:
        total_fitness = self._total_fitness_of_population()
        random_value = random() * total_fitness
        
        check_sum = 0
        selected_idx = 0
#         print(f"Random Value: {random_value}")
        while (check_sum < random_value and selected_idx < self.population_size):
#             print(f"Selected Chromosome Index: {selected_idx}, Check Sum: {check_sum}")
            check_sum += self.population[selected_idx].evaluated_score
            selected_idx += 1
#         print(f"Selected Chromosome Index: {selected_idx}, Check Sum: {check_sum}")
        return self.population[selected_idx - 1]


    def _crossover_two_parents(self) -> List[Cart]:
        parent_x = self._select_partent()
        parent_y = self._select_partent()
        while (parent_x == parent_y):
            parent_y = self._select_partent()
        children = parent_x.crossover(parent_y)
        
        return children
    
    
    def _mutate_two_children(self,mutation_prob:float,children:List[Cart]) -> List[Cart]:
        children_copy = copy.deepcopy(children)
        for child in children_copy:
            child.mutation(mutation_prob)
            
        return children_copy
    
        
    def _visualize_generation(self) -> None:
        print(f" Generation: {self.best_solution.generation}\n",f"Best Solution: \n{self.best_solution}")
        
        
    def solve(self) -> Cart:
        self._initialize_population() 
        self._visualize_generation()
        
        for _generation in range(self.num_of_generation):
            new_population = []
            for _ in range(self.population_size // 2):
                # crossover
                child_x, child_y = self._crossover_two_parents()
                # mutation
                mutated_child_x, mutated_child_y = self._mutate_two_children(self.mutation_prob, [child_x, child_y])
                new_population.extend([mutated_child_x, mutated_child_y])
                
            self.population = new_population
            self.population.sort(reverse = True)
            self.best_solution = self.population[0]
            self.global_best = max(self.global_best, self.best_solution)
            self._visualize_generation()
        
        return self.global_best
        

In [6]:
ga = GeneticAlgorithm(500,
                      0.1,
                      1000,
                      products_list,
                      3)

In [7]:
ga.solve()

 Generation: 0
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: Refrigerator C - 1199.89 - 1199.89 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 0 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1] 
Score: 24281.550000000003 
---------------------------------------------

 Generation: 1
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Produ

 Generation: 14
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Product 7: Microwave A - 308.66 - 308.66 
Product 8: Microwave B - 429.9 - 429.9 
Product 9: Microwave C - 299.29 - 299.29 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 14 
Chromosome: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1] 
Score: 24081.560000000005 
---------------------------------------------

 Generation: 15
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Produc

 Generation: 28
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Refrigerator C - 1199.89 - 1199.89 
Product 9: Notebook B - 1999.9 - 1999.9 
Product 10: Notebook C - 3999.0 - 3999.0 
Generation: 28 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1] 
Score: 23982.260000000002 
---------------------------------------------

 Generation: 29
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Produc

 Generation: 42
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: Refrigerator B - 849.0 - 849.0 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 42 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1] 
Score: 23930.660000000003 
---------------------------------------------

 Generation: 43
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product

 Generation: 55
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave C - 299.29 - 299.29 
Product 7: Refrigerator C - 1199.89 - 1199.89 
Product 8: Notebook B - 1999.9 - 1999.9 
Product 9: Notebook C - 3999.0 - 3999.0 
Generation: 55 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1] 
Score: 23542.99 
---------------------------------------------

 Generation: 56
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Refrigerator B - 849.0 - 849.0 
Product 9: Not

...

 Generation: 957
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Product 7: Microwave A - 308.66 - 308.66 
Product 8: Microwave B - 429.9 - 429.9 
Product 9: Notebook B - 1999.9 - 1999.9 
Product 10: Notebook C - 3999.0 - 3999.0 
Generation: 957 
Chromosome: [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1] 
Score: 23782.270000000004 
---------------------------------------------

 Generation: 958
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Product 7: Microwave A - 308.66 - 308.66 
Prod

 Generation: 971
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Refrigerator C - 1199.89 - 1199.89 
Product 8: Notebook B - 1999.9 - 1999.9 
Product 9: Notebook C - 3999.0 - 3999.0 
Generation: 971 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1] 
Score: 23552.36 
---------------------------------------------

 Generation: 972
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: N

 Generation: 984
 Best Solution: 
---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: Refrigerator C - 1199.89 - 1199.89 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 984 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1] 
Score: 24281.550000000003 
---------------------------------------------

 Generation: 985
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 

 Generation: 997
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Product 7: Microwave A - 308.66 - 308.66 
Product 8: Microwave B - 429.9 - 429.9 
Product 9: Microwave C - 299.29 - 299.29 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 997 
Chromosome: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1] 
Score: 24081.560000000005 
---------------------------------------------

 Generation: 998
 Best Solution: 
---------------------------------------------
Product 1: Refrigerator A - 999.9 - 999.9 
Product 2: Cell phone - 2199.12 - 2199.12 
Product 3: TV 55 - 4346.99 - 4346.99 
Product 4: TV 50'  - 3999.9 - 3999.9 
Product 5: TV 42'  - 2999.0 - 2999.0 
Product 6: Notebook A - 2499.9 - 2499.9 
Pro

---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: Refrigerator C - 1199.89 - 1199.89 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 0 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1] 
Score: 24281.550000000003 
---------------------------------------------

In [8]:
ga.best_solution

---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Refrigerator B - 849.0 - 849.0 
Product 8: Notebook B - 1999.9 - 1999.9 
Product 9: Notebook C - 3999.0 - 3999.0 
Generation: 1000 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1] 
Score: 23201.47 
---------------------------------------------

In [9]:
ga.global_best

---------------------------------------------
Product 1: Cell phone - 2199.12 - 2199.12 
Product 2: TV 55 - 4346.99 - 4346.99 
Product 3: TV 50'  - 3999.9 - 3999.9 
Product 4: TV 42'  - 2999.0 - 2999.0 
Product 5: Notebook A - 2499.9 - 2499.9 
Product 6: Microwave A - 308.66 - 308.66 
Product 7: Microwave B - 429.9 - 429.9 
Product 8: Microwave C - 299.29 - 299.29 
Product 9: Refrigerator C - 1199.89 - 1199.89 
Product 10: Notebook B - 1999.9 - 1999.9 
Product 11: Notebook C - 3999.0 - 3999.0 
Generation: 0 
Chromosome: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1] 
Score: 24281.550000000003 
---------------------------------------------